# Chapter 1: Initial Query with ChatGPT

Dear user, you are part of a consulting firm tasked to: \
[1] identify opportunities for design improvements for a specific product \
[2] conduct benchmarking with competitors whenever appropriate.

In this initial stage, we will initialize the **product** \
and gather initial insights and directions via querying a LLM (**ChatGPT**).

### TO DO SECTION

In [25]:
'''
Dear user, enter your Product here!
'''

product = "Boeing 787 Dreamliner Commercial Plane"

In [26]:
""" Use this for a fresh set up (Comment if not needed) """
import os
import shutil

if os.path.exists('support'):                     # Delete 'support' folder if it exists
    shutil.rmtree('support')

### RUN AS INTENDED (DO NOT CHANGE ANYTHING.)

##### Set Up

In [27]:
! pip install python-dotenv

In [28]:
""" Set up OpenAI API key """
import os
from dotenv import load_dotenv

load_dotenv()

key = os.getenv("OPENAI_API_KEY")

In [29]:
search_terms = product

In [30]:
""" Create directory """
import os

try:                                              # Create directory named after search terms
    os.makedirs("support/%s" % search_terms)
    print("Directory", search_terms, "created")
except FileExistsError:
    print("Directory", search_terms, "exists")

try:                                              # Create directory to store ChatGPT
    os.makedirs("support/%s/%s" % (search_terms, "ChatGPT"))
    print("Directory ChatGPT created")
except FileExistsError:
    print("Directory ChatGPT exists")

Directory Boeing 787 Dreamliner Commercial Plane created
Directory ChatGPT created


##### DESIGN OPPORTUNITIES QUERY with ChatGPT

In [38]:
""" Design Opportunities Query with ChatGPT """
import pickle
import pandas as pd
import json

from openai import OpenAI
client = OpenAI()

question =  f"Identify and rank ten opportunities for design improvements with components in {product}."
template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised explanation with a maximum of 45 characters. If you don't know the answer to the question, strictly state 'I don't know'."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"), "wb"))

data_string = pd.read_pickle("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)

try:
    components = list(data.keys())
    descriptions = list(data.values())

    df = pd.DataFrame({'Component': components, 'Description': descriptions})

    print("Design Opportunities Query with ChatGPT:")
    print(df)
except:
    print("Invalid Response. Please regenerate.")


Design Opportunities Query with ChatGPT:
                        Component                               Description
0                       Batteries  Improve safety measures to prevent fires
1                         Windows   Use self-tinting technology for comfort
2                           Seats  Enhance ergonomics for long-haul comfort
3                         Toilets         Incorporate touch-free technology
4                   Overhead Bins             Design for increased capacity
5                          Galley     Optimize layout for efficient service
6                    Landing Gear    Lightweight materials to reduce weight
7                     Wing Design  Improve aerodynamics for fuel efficiency
8                  Cabin Lighting   Use mood lighting for passenger comfort
9  In-Flight Entertainment System             Upgrade for latest technology


##### BENCHMARKING QUERY with ChatGPT

In [39]:
""" BENCHMARKING QUERY with ChatGPT """

""" Get 3 competitors """
import pickle
import pandas as pd
import json
import ast

from openai import OpenAI
client = OpenAI()

question =  f"Identify three competitors of the {product}."
template = " Express the answer only as a Python list with the product name (do not add numbering). If you don't know the answer to the question, strictly state 'I don't know'."
prompt = question + template

chat_completion = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt,}],
    temperature=0.5,                                        
)

result = chat_completion.choices[0].message.content

pickle.dump(result, open("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"), "wb"))

""" Benchmarking """
print("Benchmarking Query with ChatGPT:")
competitors_string = pd.read_pickle("support/%s/%s/chatgpt_competitors.pkl" % (product, "ChatGPT"))
competitors = ast.literal_eval(competitors_string)

data_string = pd.read_pickle("support/%s/%s/chatgpt_design_opportunities.pkl" % (product, "ChatGPT"))
data = json.loads(data_string)
components = list(data.keys())

client = OpenAI()

for i in range(len(competitors)):
    question = f"Using the {product}, conduct design benchmarking with {competitors[i]} with regards to these ten physical components: {components}."
    template = " Express the answer only as a Python dictionary (with double quotes, strictly no single quotes) with the key as a physical component (do not add numbering) and the value as a concised comparison explanation with a maximum of 40 characters. If you don't know the answer to the question, strictly state 'I don't know.'."
    prompt = question + template

    chat_completion = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,  # adjust persona and temperature
    )

    result = chat_completion.choices[0].message.content

    pickle.dump(result, open("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]), "wb"))

    data_string = pd.read_pickle("support/%s/%s/chatgpt_%s_benchmarking.pkl" % (product, "ChatGPT", competitors[i]))
    benchmarking_data = json.loads(data_string)
    try:
        components = list(benchmarking_data.keys())
        descriptions = list(benchmarking_data.values())

        df = pd.DataFrame({'Component': components, 'Benchmarking': descriptions})

        print(competitors[i])
        print(df)
        print("\n")
    except:
        print("Invalid Response. Please regenerate.")

Benchmarking Query with ChatGPT:
Airbus A350 XWB
                        Component                         Benchmarking
0                       Batteries   787 had battery overheating issues
1                         Windows           787's are larger, dimmable
2                           Seats     Depends on airline configuration
3                         Toilets  A350's are larger, more comfortable
4                   Overhead Bins         787's are larger, more space
5                          Galley      A350 has larger, more efficient
6                    Landing Gear            Similar, depends on model
7                     Wing Design  787's more flexible, fuel efficient
8                  Cabin Lighting              Both have mood lighting
9  In-Flight Entertainment System            Depends on airline choice


Airbus A330neo
                        Component                                Benchmarking
0                       Batteries       787 uses lithium-ion, A330 uses Ni-